<a href="https://colab.research.google.com/github/modinitin13/Image-caption-generator/blob/master/Image%20caption%20generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from numpy import array
import pickle

In [0]:
import shutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import os
from os import listdir
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from pickle import dump
from keras.applications.vgg16 import VGG16
from keras.layers import Input

from keras.models import Model


In [0]:
  in_layer = Input(shape=(224, 224, 3))
  model = VGG16(include_top=False, input_tensor=in_layer, pooling='avg')
  model.summary()

In [0]:
def extract_features(directory):
  count = 0
  features= dict()
      
  in_layer = Input(shape=(224, 224, 3))
  model = VGG16(include_top=False, input_tensor=in_layer, pooling='avg')
  print(model.summary())
  for image_name in listdir(directory):
    filename = directory + "/" + image_name
    image=load_img(filename, target_size=(224,224))
    image=img_to_array(image)
    image = image.reshape(1, image.shape[0], image.shape[1], image.shape[2])
    image=preprocess_input(image)
    feature = model.predict(image, verbose=0)
    image_id= image_name.split(".")[0]
    features[image_id]=feature
    print(">>> "+ image_id)
  print("done")
  return features

directory = "drive/My Drive/Flicker8k_Data/Flicker8k_Dataset"

features= extract_features(directory)
print(len(features))

dump(features, open('features.pkl', 'wb'))
  
  
  
  
  
  
  
  

In [0]:
import pickle
with open('features.pkl', 'rb') as f:
    features = pickle.load(f)

In [0]:
def load_doc(filename):
	file = open(filename, 'r')
	text = file.read()
	file.close()
	return text

In [0]:
import pickle
with open('features.pkl', 'wb') as handle:
    pickle.dump(features, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
def load_description(doc):
  descriptions=dict()
  for line in doc.split("\n"):
    tokens = line.split()
    if (len(line)<2):
      continue
    image_id, img_desc = tokens[0], tokens[1:]
    image_id=image_id.split('.')[0]
    img_desc=' '.join(img_desc)
    if image_id not in descriptions:
      descriptions[image_id]=list()
    descriptions[image_id].append(img_desc)
  return descriptions


In [0]:
filename = "Flickr8k.token.txt"
doc = load_doc(filename)
full_descriptions = load_description(doc)
print(len(full_descriptions))

8092


In [0]:
def clean_descriptions(descriptions):
  import string
  table = str.maketrans('', '', string.punctuation)
  for key, desc_list in descriptions.items():
    for i in range(len(desc_list)):
      desc=desc_list[i].split()
      desc = [word.lower() for word in desc]
      desc = [w.translate(table) for w in desc]
      desc = [word for word in desc if len(word)>1 and word.isalpha()]
      descriptions[key][i]= ' '.join(desc)
  return descriptions
      
    
  

In [0]:
descriptions

In [0]:
def save_doc(descriptions, filename):
  lines =list()
  for keys, desc in descriptions.items():
    for d in desc:     
      lines.append(keys + " " + d)
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()
    
  

In [0]:
save_doc(descriptions, 'descriptions.txt')

In [0]:
descriptions= clean_descriptions(descriptions)

In [0]:
def to_vocabulary(descriptions):
  all_words=set()
  for keys in descriptions.keys():
    for desc in descriptions[keys]:
      [all_words.update(desc.split())]
  return all_words
      

In [0]:
vocabulary = to_vocabulary(train_descriptions)

In [0]:
voc

In [0]:
vocab_len

7578

In [0]:
def load_set(filename):
  doc = load_doc(filename)
  dataset = list()
  for line in doc.split('\n'):
    if (len(line)<1):
      continue
    dataset.append(line.split('.')[0])
  return set(dataset)
  

In [0]:
def load_clean_desc(filename, dataset):
  doc = load_doc(filename)
  description = dict()
  for line in doc.split('\n'):
    tokens = line.split()
    image_id, img_desc = tokens[0], tokens[1:]
    if image_id not in dataset:
      continue
    if image_id in dataset:
      if image_id not in description:
        description[image_id] = list()
        desc = 'startseq ' + ' '.join(img_desc) + " endseq"
        description[image_id].append(desc)
  return description
    

In [0]:
def load_photo_features(filename, dataset):
  
  with open(filename, 'rb') as f:
    all_features = pickle.load(f)
  features =  {k:all_features[k] for k in dataset}
  return features
  
  

In [0]:
from pickle import load
filename = 'drive/My Drive/Flicker8k_Data/Flickr8k_text/Flickr_8k.trainImages.txt'
train_set = load_set(filename)
print('Dataset: %d' % len(train_set))
train_descriptions = load_clean_desc('descriptions.txt', train_set)
print('Descriptions: train=%d' % len(train_descriptions))
train_features = load_photo_features('features.pkl', train_set)
print('Photos: train=%d' % len(train_features))



Dataset: 6000
Descriptions: train=6000
Photos: train=6000


In [0]:
train_features['1000268201_693b08cb0e'][0]

In [0]:
filename = 'drive/My Drive/Flicker8k_Data/Flickr8k_text/Flickr_8k.devImages.txt'
dev_set = load_set(filename)
print('Dataset: %d' % len(dev_set))
dev_descriptions = load_clean_desc('descriptions.txt', dev_set)
print('Descriptions: train=%d' % len(dev_descriptions))
dev_features = load_photo_features('features.pkl', dev_set)
print('Photos: train=%d' % len(dev_features))


Dataset: 1000
Descriptions: train=1000
Photos: train=1000


In [0]:
def to_lines(descriptions):
  all_desc = list()
  for key in descriptions.keys():
    [all_desc.append(d) for d in descriptions[key]]
  return all_desc

def create_tokenizer(descriptions):
  lines = to_lines(descriptions)
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(lines)
  return tokenizer

In [0]:
tokenizer = create_tokenizer(train_descriptions)

In [0]:
vocab_len = len(tokenizer.word_index)+1

In [0]:
vocab_len

3848

In [0]:
ddd=train_descriptions['1000268201_693b08cb0e'][0]

In [0]:
dump(tokenizer, open('tokenizer.pkl', 'wb'))

In [0]:
all_desc = to_lines(train_descriptions)

In [0]:
train_descriptions

In [0]:
all_set= train_set|dev_set
all_descriptions = load_clean_desc('descriptions.txt', all_set)

In [0]:
all_descriptions

In [0]:
def find_max_len(desc):
  max=0
  for key in desc.keys():
    for sent in desc[key]:
      if (max<len(sent.split(" "))):
        max = len(sent.split(" "))
  return max

max_len = find_max_len(desc=all_descriptions)
print(max_len)

30


In [0]:
bbb=tokenizer.texts_to_sequences(ddd)

In [0]:
train_features['1084104085_3b06223afe'].shape

(1, 512)

In [0]:
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

In [0]:
max_length(all_descriptions)

30

In [0]:
vocab_len = len(tokenizer.word_index) + 1

In [0]:
def model(vocab_len, max_len):
  inputs1 = Input(shape = (512,))
  fe1 = Dropout(0.5)(inputs1)
  fe2 = Dense(256, activation='relu')(fe1)
  
  inputs2 = Input(shape = (max_len,))

  se1 = Embedding(vocab_len, 256, mask_zero=True)(inputs2)    #mask_zero=true for padding .... 0 will be for padding
  se2 = Dropout(0.5)(se1)
  se3 = LSTM(256, activation = 'relu')(se2)
  
  decoder1 = add([fe2, se3])
  decoder2 = Dense(256, activation = 'relu')(decoder1)
  outputs = Dense(vocab_len, activation = 'softmax')(decoder2)
  
  model = Model(inputs = [inputs1, inputs2], outputs=outputs)
  model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
  print(model.summary())
  
  plot_model(model, to_file='model.png', show_shapes=True)
  return model

In [0]:
filepath = 'drive/My Drive/weights/image_caption/model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [0]:
model= model(vocab_len, max_len)

In [0]:
def create_sequences(tokenizer, max_len, descriptions, image_features, vocab_len):
  X1, X2, Y = list(), list(), list()
  for key, description in descriptions.items():

    for each_desc in description:
      seq = tokenizer.texts_to_sequences([each_desc])[0]
      for i in range(1, len(seq)):
        in_seq, out_seq = seq[:i], seq[i]
        in_seq = pad_sequences([in_seq], maxlen=max_len)[0]
        out_seq = to_categorical([out_seq], num_classes = vocab_len)[0]
        X1.append(image_features[key])
        X2.append(in_seq)
        Y.append(out_seq)
  return array(X1).reshape(-1,512), array(X2).reshape(-1,30), array(Y)
  

In [0]:
ytrain.shape

(61308, 3848)

In [0]:
X1train, X2train, ytrain = create_sequences(tokenizer, max_len, descriptions=train_descriptions, image_features=train_features, vocab_len=vocab_len)
X1test, X2test, ytest = create_sequences(tokenizer, max_len, descriptions=dev_descriptions, image_features=dev_features, vocab_len=vocab_len)


In [0]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [0]:
model.fit([X1train, X2train], ytrain, epochs=20, verbose=1, callbacks=[checkpoint, learning_rate_reduction], validation_data=([X1test, X2test], ytest))

# **Sampling**

In [0]:
def encoder_model():
  in_layer = Input(shape=(224, 224, 3))
  model = VGG16(include_top=False, input_tensor=in_layer, pooling='avg')
  

In [0]:
def extract_features(filenames, model):
  count = 0
  features= list()
      
  print(model.summary())
  for filename in file_names:
    image=load_img(filename, target_size=(224,224))
    image=img_to_array(image)
    image = image.reshape(1, image.shape[0], image.shape[1], image.shape[2])
    image=preprocess_input(image)
    feature = model.predict(image, verbose=0)
    features.append(feature)
    print(">>> "+ image_id)
  print("done")
  print(len(features))
  return features

In [0]:
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

In [0]:
def generate_desc(model, tokenizer, features, max_len):
	in_text = 'startseq'
	for i in range(max_length):
		sequence = tokenizer.texts_to_sequences([in_text])[0]
		sequence = pad_sequences([sequence], maxlen=max_length)
		yhat = model.predict([photo,sequence], verbose=0)
		yhat = argmax(yhat)
		word = word_for_id(yhat, tokenizer)
		if word is None:
			break
		in_text += ' ' + word
		if word == 'endseq':
			break
	return in_text


In [0]:
from nltk.translate.bleu_score import corpus_bleu
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
	actual, predicted = list(), list()
	for key, desc_list in descriptions.items():
		yhat = generate_desc(model, tokenizer, photos[key], max_length)
		references = [d.split() for d in desc_list]
		actual.append(references)
		predicted.append(yhat.split())
	print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
	print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
	print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
	print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

In [0]:
filename = 'drive/My Drive/Flicker8k_Data/Flickr8k_text/Flickr_8k.testImages.txt'
test = load_set(filename)
print('Dataset: %d' % len(test))
# descriptions
test_descriptions = load_clean_descriptions('descriptions.txt', test)
print('Descriptions: test=%d' % len(test_descriptions))
# photo features
test_features = load_photo_features('features.pkl', test)
print('Photos: test=%d' % len(test_features))

# evaluate model
evaluate_model(model, test_descriptions, test_features, tokenizer, max_length)